# Federated learning with pysyft on MNIST data
in this notebook, we are going to cover training a neural network on the MNIST dataset while implementing the federated learning approach with the pysyft library.

## prerequisites
- Familiarity with pysyft [check this notebook on an introductory guide to pysyft](https://jovian.ai/tifeasypeasy/introducing-privacy-preserving-tool)

## Federated learning workflow

 - Edge devices  receives a copy of a global model from a central server.
 - The model is being trained locally on the data residing on the edge devices 
 - The global model weights are updated during training on each worker
 - A local copy is sent back to the central server 
 - The server receives various updated model and aggregate the updates thereby improving the global model and also preserving privacy of data in which it was being trained on.


![image](https://miro.medium.com/max/640/0*yI_rIRNAFTYDwFtK.png)

Figure1. Federated learning

Use cases
- **Next word prediction**: Federated learning is used to improve word prediction in mobile devices without uploading users data to the cloud for training. Google gboard implements FL by using on-device data to train and improve the gboard next word prediction model for it's users, watch this video to better understand how google uses federated learning at scale. You can also read this online comic from Google AI to get a better grasp of federated learning. 

- **Voice recognition**: FL is also used in voice recognition technologies, an example is in apple Siri and just recently Google introduces the audio recognition technology on Google assistant to train and better improve users experience with the Google assistant. Watch a demonstration of FL on audio recording for speech systems here. 

In every of these use cases, the data doesn't leave the edge devices thereby keeping the data private, safe and secure while still improving these technologies and making products smarter over time.

# Setup

## How to run the code 
Running this notebook is the same as running the 'introducing-privacy-preserving-tool' notebook. You can either run it online on the jovian platform [via this link](https://jovian.ai/tifeasypeasy/federated-learning-on-mnist) without having to install anything or locally on your machine 


In [1]:
# !pip install jovian --upgrade --quiet
# uncomment the above command if youre running on binder

In [2]:
# this installs the syft library, comment the below command if you were able to install syft locally
!pip install syft

     |████████████████████████████████| 440kB 3.3MB/s eta 0:00:01
     |████████████████████████████████| 1.8MB 26.0MB/s eta 0:00:01
     |████████████████████████████████| 184kB 20.2MB/s eta 0:00:01
     |████████████████████████████████| 753.4MB 24kB/s  eta 0:00:012   |                                | 2.5MB 13.7MB/s eta 0:00:55     |▏                               | 3.9MB 13.7MB/s eta 0:00:552322     |█████▊                          | 133.7MB 48.0MB/s eta 0:00:13     |█████▊                          | 135.0MB 48.0MB/s eta 0:00:13MB/s eta 0:00:13     |██████                          | 143.4MB 48.0MB/s eta 0:00:134MB 34.8MB/s eta 0:00:17     |████████▎                       | 194.6MB 24.6MB/s eta 0:00:23     |██████████▏                     | 238.4MB 50.4MB/s eta 0:00:11          | 348.2MB 33.7MB/s eta 0:00:13     |███████████████▉                | 372.5MB 38.0MB/s eta 0:00:11     |████████████████                | 376.2MB 38.0MB/s eta 0:00:10     |██████████████████▍             | 43

     |████████████████████████████████| 81kB 10.2MB/s eta 0:00:01
     |████████████████████████████████| 133kB 17.1MB/s eta 0:00:01
     |████████████████████████████████| 204kB 25.4MB/s eta 0:00:01
     |████████████████████████████████| 2.0MB 34.7MB/s eta 0:00:01
     |████████████████████████████████| 81kB 25.3MB/s eta 0:00:01
     |████████████████████████████████| 71kB 24.1MB/s eta 0:00:01
     |████████████████████████████████| 61kB 20.6MB/s eta 0:00:01
     |████████████████████████████████| 307kB 42.8MB/s eta 0:00:01
     |████████████████████████████████| 36.9MB 777kB/s eta 0:00:011   | 1.2MB 21.0MB/s eta 0:00:02��██████▍              | 20.1MB 21.0MB/s eta 0:00:01.5MB 21.0MB/s eta 0:00:01B/s eta 0:00:01█▊             | 21.6MB 21.0MB/s eta 0:00:01█████████████             | 22.0MB 21.0MB/s eta 0:00:01��█████████▍            | 22.4MB 21.0MB/s eta 0:00:01
     |████████████████████████████████| 81kB 14.6MB/s eta 0:00:01
     |████████████████████████████████| 51kB 20.0MB/s eta 0

     |████████████████████████████████| 51kB 20.1MB/s eta 0:00:01
  Created wheel for dill: filename=dill-0.3.2-cp37-none-any.whl size=78912 sha256=6e8290b4ed0494e004c60bcf8c1455487f888bf1da77aeca4be19d965957a16a
  Stored in directory: /home/jovyan/.cache/pip/wheels/27/4b/a2/34ccdcc2f158742cfe9650675560dea85f78c3f4628f7daad0
  Created wheel for phe: filename=phe-1.4.0-py2.py3-none-any.whl size=37361 sha256=fd08b616e8c52ca979b21387a82373c9c339ab42600a56684c37e2bb5d3e02f2
  Stored in directory: /home/jovyan/.cache/pip/wheels/f8/dc/36/dcb6bf0f1b9907e7b710ace63e64d08e7022340909315fdea4
  Created wheel for tornado: filename=tornado-4.5.3-cp37-cp37m-linux_x86_64.whl size=434105 sha256=8b2fcb53f44e5f96e1b52c0631101f6bb2e13d266b6e7bed6d9eda734a2ca4cb
  Stored in directory: /home/jovyan/.cache/pip/wheels/72/bf/f4/b68fa69596986881b397b18ff2b9af5f8181233aadcc9f76fd
  Created wheel for psutil: filename=psutil-5.7.0-cp37-cp37m-linux_x86_64.whl size=276461 sha256=7d913986e39dacddc060b4ee5c244e7673f0

## Imports and initializing hook
Lets import torch, torchvision, and other modules 

In [21]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [15]:
import syft as sy  # import the syft library
hook = sy.TorchHook(torch)  # attach the pytorch hook
joe = sy.VirtualWorker(hook, id="joe")  #  remote worker joe
jane = sy.VirtualWorker(hook, id="jane")  #  remote worker  jane

## Load in the MNIST dataset 
Let's load in the data and transform it into a federated dataset by implementing the federate() method.

In [16]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, ), (0.5, )),
])

train_set = datasets.MNIST(
    "~/.pytorch/MNIST_data/", train=True, download=True, transform=transform)
test_set = datasets.MNIST(
    "~/.pytorch/MNIST_data/", train=False, download=True, transform=transform)

federated_train_loader = sy.FederatedDataLoader(
    train_set.federate((joe, jane)), batch_size=64, shuffle=True) # the federate() method splits the data within the workers

test_loader = torch.utils.data.DataLoader(
    test_set, batch_size=64, shuffle=True)


## Define the network architecture
The network architecture would remain the same just as the example tutorial from pytorch with an input of 784-dim tensor of pixel values for each image, and producing a tensor of length 10  which indicates the class scores for an input image

In [17]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(784, 500)
        self.fc2 = nn.Linear(500, 10)

    def forward(self, x):
        x = x.view(-1, 784)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)
model = Net()
print(model)
optimizer = optim.SGD(model.parameters(), lr=0.01)

Net(
  (fc1): Linear(in_features=784, out_features=500, bias=True)
  (fc2): Linear(in_features=500, out_features=10, bias=True)
)


## Train the network 
Looking at the training process of this distributed approach, the data seem to be on a remote machine so therefore we would use the location attribute to get the location and send our model to that location where the data is present, we would then get back the improved model using the get() method and calculate the loss. 


In [18]:
n_epoch = 10 
for epoch in range(n_epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(federated_train_loader):
        model.send(data.location) # send the model to the client device where the data is present
        optimizer.zero_grad()         # training the model
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        model.get() # get back the improved model
        if batch_idx % 100 == 0: 
            loss = loss.get() # get back the loss
            print('Training Epoch: {:2d} [{:5d}/{:5d} ({:3.0f}%)]\tLoss: {:.6f}'.format(
                epoch+1, batch_idx * 64,
                len(federated_train_loader) * 64,
                100. * batch_idx / len(federated_train_loader), loss.item()))

Training Epoch:  1 [    0/60032 (  0%)]	Loss: 2.347703
Training Epoch:  1 [ 6400/60032 ( 11%)]	Loss: 1.378724
Training Epoch:  1 [12800/60032 ( 21%)]	Loss: 0.804508
Training Epoch:  1 [19200/60032 ( 32%)]	Loss: 0.622711
Training Epoch:  1 [25600/60032 ( 43%)]	Loss: 0.551721
Training Epoch:  1 [32000/60032 ( 53%)]	Loss: 0.462930
Training Epoch:  1 [38400/60032 ( 64%)]	Loss: 0.465403
Training Epoch:  1 [44800/60032 ( 75%)]	Loss: 0.512338
Training Epoch:  1 [51200/60032 ( 85%)]	Loss: 0.550041
Training Epoch:  1 [57600/60032 ( 96%)]	Loss: 0.540856
Training Epoch:  2 [    0/60032 (  0%)]	Loss: 0.484230
Training Epoch:  2 [ 6400/60032 ( 11%)]	Loss: 0.346220
Training Epoch:  2 [12800/60032 ( 21%)]	Loss: 0.197471
Training Epoch:  2 [19200/60032 ( 32%)]	Loss: 0.405758
Training Epoch:  2 [25600/60032 ( 43%)]	Loss: 0.270681
Training Epoch:  2 [32000/60032 ( 53%)]	Loss: 0.368734
Training Epoch:  2 [38400/60032 ( 64%)]	Loss: 0.467331
Training Epoch:  2 [44800/60032 ( 75%)]	Loss: 0.277411
Training E

## Testing the trained model
Remember the test dataset remains unchanged as it is on our local machine compared to the train dataset which we have splitted between two virtual workers.

In [19]:
model.eval()
test_loss = 0
correct = 0
with torch.no_grad():
    for data, target in test_loader:
        output = model(data)
        test_loss += F.nll_loss(
            output, target, reduction='sum').item()
        pred = output.argmax(1, keepdim=True) # get the index of the max log-probability
        correct += pred.eq(target.view_as(pred)).sum().item()

test_loss /= len(test_loader.dataset)

print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
    test_loss,correct,
    len(test_loader.dataset),
    100. * correct / len(test_loader.dataset)))


Test set: Average loss: 0.1762, Accuracy: 9475/10000 (95%)



As you can see, we achieved an accuracy of 95% which is pretty good for a federated sytem as this tutorial has demonstrated. 